Here's a step-by-step guide to build, run, and push your Docker app, including the configuration for accessing the live camera feed.

### Step 1: Set Up Your Project Directory

1. **Create the project directory:**
   ```sh
   mkdir LicensePlateRecognizer
   cd LicensePlateRecognizer
   ```

2. **Place your Python files and models in the directory:**
   - `app.py` (Main application entry)
   - `License_Plate_Recognizer.py` (Your recognizer class)
   - `ANPR.py` (Any additional scripts)
   - `saved_model/` (Your pre-trained models)
   - `requirements.txt` (Python dependencies)

3. **Create a `Dockerfile`** in the project directory.

### Step 2: Create the `Dockerfile`

Your `Dockerfile` should look like this:

```Dockerfile
# Use the official Python image as a base
FROM python:3.8-slim

# Set environment variables to prevent Python from buffering stdout/stderr
ENV PYTHONUNBUFFERED=1

# Set the working directory in the container
WORKDIR /app

# Install the necessary dependencies
RUN apt-get update && apt-get install -y \
    libgl1-mesa-glx \
    libglib2.0-0 \
    && rm -rf /var/lib/apt/lists/*

# Install Python dependencies
COPY requirements.txt /app/requirements.txt
RUN pip install --no-cache-dir -r /app/requirements.txt

# Copy the application code and pre-trained models into the container
COPY . /app

# Set the default command to run the app
CMD ["python", "app.py"]
```

### Step 3: Create the `requirements.txt`

Make sure your `requirements.txt` includes all necessary Python packages:

```
opencv-python-headless
torch
ultralytics
transformers
Pillow
```

### Step 4: Build the Docker Image

In your terminal, navigate to the project directory and build the Docker image:

```sh
docker build -t license-plate-recognizer .
```

This command will build a Docker image named `license-plate-recognizer` using the `Dockerfile` in your current directory.

### Step 5: Run the Docker Container

#### For Processing a Video or Image

If you want to process an image or video, you can pass the paths as arguments:

- **Image**:
  ```sh
  docker run --rm -v $(pwd):/app license-plate-recognizer --image /app/image_3.png
  ```

- **Video**:
  ```sh
  docker run --rm -v $(pwd):/app license-plate-recognizer --video /app/video.mp4
  ```

Here, `-v $(pwd):/app` mounts your current directory into the container, allowing it to access files on your host machine.

#### For Live Camera Feed (Linux)

To run the live camera feed on Linux, pass the video device:

```sh
docker run --rm --device=/dev/video0 license-plate-recognizer
```

This command gives the container access to your webcam.

### Step 6: Push the Docker Image to a Repository

If you want to push your Docker image to a container registry like Docker Hub, follow these steps:

1. **Login to Docker Hub:**
   ```sh
   docker login
   ```

2. **Tag the Docker Image:**
   ```sh
   docker tag license-plate-recognizer your-dockerhub-username/license-plate-recognizer:latest
   ```

3. **Push the Image to Docker Hub:**
   ```sh
   docker push your-dockerhub-username/license-plate-recognizer:latest
   ```

Replace `your-dockerhub-username` with your actual Docker Hub username.

### Step 7: Running the Container on Another Machine

Once the image is pushed to Docker Hub, you can pull and run it on any machine:

1. **Pull the Docker Image:**
   ```sh
   docker pull your-dockerhub-username/license-plate-recognizer:latest
   ```

2. **Run the Container:**
   ```sh
   docker run --rm -v $(pwd):/app your-dockerhub-username/license-plate-recognizer:latest --image /app/image_3.png
   ```

Or, for a live feed on Linux:

```sh
docker run --rm --device=/dev/video0 your-dockerhub-username/license-plate-recognizer:latest
```

This guide walks you through building, running, and pushing a Docker container that can process images, videos, and live camera feeds for license plate recognition.

In [1]:
from License_Plate_Recognizer import LicensePlateRecognizer

recognizer=LicensePlateRecognizer()
recognizer.load_model()


***YOLOv8 Model Successfully Loaded and Moved to cpu***


***OCR Model Successfully Loaded and Moved to cuda:0***



In [2]:
%%time
image_path='image_4.png'

recognizer.process_image(image_path)

CPU times: total: 9.44 s
Wall time: 14.1 s


'MP 09BD 0122'

In [6]:
%%time
video_path='192.168.112.13_01_20240821162656542.mp4'

recognizer.process_video(video_path, frame_skip=50)

Frames per second (FPS): 25
At :10:34:33,
 The plate number: AP 04BC 5618 detected with confidence: 0.7273319363594055
At :10:34:37,
 The plate number: AP 04BC 5618 detected with confidence: 0.7139596939086914
At :10:34:41,
 The plate number: AP 04CG 3406 detected with confidence: 0.3262089788913727
At :10:34:44,
 The plate number: AP 04CG 3406 detected with confidence: 0.7920824885368347
At :10:34:48,
 The plate number: AP 04CG 3406 detected with confidence: 0.6475366353988647
At :10:34:51,
 The plate number: AP 04CG 3406 detected with confidence: 0.6232821345329285
At :10:34:55,
 The plate number: AP 04CG 3406 detected with confidence: 0.5060810446739197
At :10:34:59,
 The plate number: AP 04CG 3406 detected with confidence: 0.466504842042923
At :10:35:02,
 The plate number: AP 04CG 3406 detected with confidence: 0.5014327764511108
At :10:35:06,
 The plate number: AP 04CG 3406 detected with confidence: 0.3752710521221161
At :10:35:10,
 The plate number: AP 04CG 3406 detected with con

In [ ]:

# Your existing code...
def main():
    recognizer=LicensePlateRecognizer()
    recognizer.load_model()
    while True:
        image_path = input("\nEnter the image path (or type 'exit' to quit): ")
        if image_path.lower() == 'exit':
            print("Exiting...")
            return
        if not os.path.exists(image_path):
            print(f"Error: The file {image_path} does not exist.")
            continue
        try:
            plate_number, processed_image = recognizer.process_image(image_path)
            # Ensure the 'output' directory exists in the current working directory
            output_dir = os.path.join(os.getcwd(), "output")
            os.makedirs(output_dir, exist_ok=True)

            # Create the output file path
            output_image_path = os.path.join(output_dir, f"output_{plate_number}.jpg")

            # Save the processed image
            cv2.imwrite(output_image_path, processed_image)

            print(f"Recognized License Plate: {plate_number}")
            print(f"Processed image saved as {output_image_path}")
        except:
            print('No plate found')

if __name__ == "__main__":
    main()